## GeoUtils with NetCDF, VSIS3, and earthaccess

In [ ]:
import rasterio as rio
from geoutils import Raster

In [ ]:
def configure_auth():
    import boto3
    import earthaccess
    from rasterio.session import AWSSession

    auth = earthaccess.login()
    s3_credentials = auth.get_s3_credentials("PODAAC")
    session = boto3.Session(
        aws_access_key_id=s3_credentials["accessKeyId"],
        aws_secret_access_key=s3_credentials["secretAccessKey"],
        aws_session_token=s3_credentials["sessionToken"],
        region_name="us-west-2",
    )
    rio_env = rio.Env(
        AWSSession(session),
    )
    rio_env.__enter__()
    return rio_env

In [ ]:
def load_data():
    bucket = "podaac-ops-cumulus-protected"
    input_uri = "MUR-JPL-L4-GLOB-v4.1/20020601090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc"
    src = f"NETCDF:/vsis3/{bucket}/{input_uri}:analysed_sst"
    da = rio.open(src)
    da = Raster(da)
    return da.load()

In [ ]:
if __name__ == "__main__":
    rio_env = configure_auth()
    da = load_data()
    rio_env.__exit__()